In [347]:
# ブログから画像を抽出するやつ
import requests # urlを読み込むためrequestsをインポート
from bs4 import BeautifulSoup # htmlを読み込むためBeautifulSoupをインポート
import os
import re
import time
import numpy as np
import urllib
import datetime


In [348]:
def set_Name_of_the_Menber():

    MenberList_kanji = ['石森 虹花','今泉 佑唯', '上村 莉菜','尾関 梨香','織田 奈那', '小池 美波','小林 由依', '齋藤 冬優花','佐藤 詩織', '志田 愛佳','菅井 友香','鈴本 美愉', '長沢 菜々香','長濱 ねる','土生 瑞穂', '原田 葵', '平手 友梨奈', '守屋 茜', '米谷 奈々未', '渡辺 梨加', '渡邉 理佐']
    MenberList_hiragana = ['井口 眞緒','潮 紗理菜', '柿崎 芽実','影山 優佳','加藤 史帆', '齊藤 京子','佐々木 久美', '佐々木 美玲','高瀬 愛奈', '高本 彩花','東村 芽依','金村 美玖', '河田 陽菜','小坂 菜緒','富田 鈴花', '丹生 明里', '濱岸 ひより', '松田 好花', '宮田 愛萌', '渡邉 美穂']

    MenberList = Menber_kanji + Menber_hiragana
    
    n = 0;
    for i in MenberList:
        print(str(n) +': '+ MenberList[n])
        n += 1
        
    n = input('Choose a number \n')
    print(MenberList[int(n)])

    return MenberList_kanji,MenberList_hiragana,str(MenberList[int(n)])


In [349]:
def get_Number_of_LatestBlog():
    URL  = 'http://www.keyakizaka46.com/s/k46o/?ima=0000'
    soup = BeautifulSoup(requests.get(URL).content,'lxml') # bsでURL内を解析
    links = [a.get("href") for a in soup.find_all("a")]
#     print(links)

    # topページからblogのリンクのみを抽出、1つ目が細心なのでbreak
    for i in range(len(links)):
        if ('/s/k46o/diary/detail/' in links[i]) == True:
            new_blog =links[i]
            break

    
    # ブログ番号の抽出
    arr = new_blog.split('/')
    new_blog_num = arr[5].split('?')[0]
#     print(new_blog)
#     print(arr)
#     print(new_blog_num)
    return int(new_blog_num)

In [411]:
def get_Number_of_Blog(kanji,hiragana):

    MenberList = kanji + hiragana
    
#　辞書リストでブログ番号とメンバーを紐付けする
    list_number_of_the_blogs={}
    for i in MenberList:
        list_number_of_the_blogs[i] =[]
    
    latest_blog_num = get_Number_of_LatestBlog()

    print('Latest blog number :' + str(latest_blog_num))
    url_front = 'http://www.keyakizaka46.com/s/k46o/diary/detail/'
    url_back = '?ima=0000&cd=member'

    start_blog_num = 1
    latest_blog_num -= 1;
#     latest_blog_num = 10030

    
    n = latest_blog_num - start_blog_num 
    
    for i in range(n):
        blog_number = str(i+start_blog_num)
    
        # ブログのurl の作成する
        url_total = url_front + blog_number + url_back
        
        # URLが有効か確かめる error : 無視する
        try:
            urllib.request.urlopen(url_total)
        except urllib.error.HTTPError as e:
#             print("This URL Not Found")
#             break

        # サーバーの負荷対策
         time.sleep(1)
         i += 1

        soup = BeautifulSoup(requests.get(url_total).content,'lxml') # bsでURL内を解析
        title = soup.find("title")
        arr = title.string.split('公式ブログ')

        blog_author = arr[0]

#         return blog_author == name            
        
            
        for j in range(len(MenberList)):
            if MenberList[j] == blog_author:  # このif文に時間がかかっていそう
                list_number_of_the_blogs[MenberList[j]].append(blog_number)
                break
    
        if i%1000 ==0:
            print('Latest blog number :' + str(blog_number))
            make_txt_file(list_number_of_the_blogs,blog_number)
    
    return list_number_of_the_blogs,latest_blog_num


In [412]:
def check_Menber_of_this_Blog(name,blog_url):
    
    soup = BeautifulSoup(requests.get(blog_url).content,'lxml') # bsでURL内を解析
    
    # titleタグの取得　ページ内にtitleタグは1つだけなので、これで十分
    # .string でタグ内の文字列を取得
    # splitでtitleから名前のみを抜き出す
    title = soup.find("title")
    arr = title.string.split('公式ブログ')

    blog_author = arr[0]

    return blog_author == name
    

In [413]:
def make_txt_file(list,num):
    filename = 'list' + str(num) + '.txt'
    print(filename)
    
    f = open(filename, 'w')       # ファイルを開く(該当ファイルがなければ新規作成)
    for x in list:                          # 文字列を記載する
#         print(x)
#         print(list[x])
        f.write(str(x) + str(list[x]) + "\n")
    
    f.close()                                #ファイルを閉じる


In [414]:
def make_folder_for_save(Menber):
    path = os.getcwd()
    day = datetime.date.today()#今日の日付
    time = datetime.datetime.today()#時間まで
    os.mkdir(str(Menber) + str(day))
    folder = str(Menber) + str(day)
    
    return path
    

In [438]:
# ブログから画像を抽出するやつ
def GetPicture(Blog_Number_List,Menber):
    images = []
    pic = []
    blog_num = []
    path = make_folder_for_save(Menber)
    
    _path = os.getcwd()
    path = _path + '/'  + str(Menber)
    os.mkdir(str(Menber))

    url_front = 'http://www.keyakizaka46.com/s/k46o/diary/detail/'
    url_back = '?ima=0000&cd=member'

    for n in range(len(Blog_Number_List)):
            blog_num.append(Blog_Number_List[n])
    
    for i in blog_num:
        images = []
        pic = []
        url_total = url_front + str(i) + url_back
        print(url_total)
        soup = BeautifulSoup(requests.get(url_total).content,'lxml') # bsでURL内を解析

        for link in soup.find_all("img"): # imgタグを取得しlinkに格納
            if link.get("src").endswith(".jpg"): # imgタグ内の.jpgであるsrcタグを取得
                images.append(link.get("src")) # imagesリストに格納
            elif link.get("src").endswith(".png"): # imgタグ内の.pngであるsrcタグを取得
                images.append(link.get("src")) # imagesリストに格納
        # １ブログページに元からある画像は64個
        #　１番目は欅ロゴマークなので、２番目以降にブログ画像が保存される
        # ブログ画像の数は、len(images)-64　個
        num = len(images) - 64

        # ブログ画像のみの配列を作り直す

        
        for j in range(num):
            pic.append(images[j+1])
        
        pic_num = len(pic)
        print(pic_num)
        
    # 画像を保存する際にリネームする
        m = 1
        for target in pic: # picからtargetに入れる
            print(m)
            
            re = requests.get(target)
#             print(target)
            save_path= str(path) + '/' + target.split('/')[-1]
#             print(save_path)
            with open(save_path, 'wb') as f: # imgフォルダに格納
                f.write(re.content) # .contentにて画像データとして書き込む
                os.chdir(str(Menber))
                
                if pic_num == 1:  
                    os.rename(save_path,str(Menber)+'_'+str(i)+'.png' )
                elif pic_num > 1:
                    os.rename(save_path,str(Menber)+'_'+str(i) + '_' + str(m) +'.png' )
            
                os.chdir('../')
                m = m +1
                time.sleep(1)



In [445]:
# blog_list = {'今泉 佑唯':['63', '77', '93', '98', '121', '148', '155', '158', '177']}
# blog_list = {'今泉 佑唯':['63', '77', '93', '98', '121', '148', '155', '158', '177', '194', '206', '233', '250', '263', '279', '298', '316', '336', '349', '372', '382', '404', '409', '436', '450', '468', '477', '488', '507', '529', '543', '562', '572', '594', '602', '621', '644', '661', '678', '701', '714', '727', '754', '770', '781', '798', '816', '839', '851', '869', '883', '894', '901', '917', '940', '952', '967', '981', '1001', '1018', '1032', '1050', '1063', '1077', '1098', '1106', '1127', '1138', '1152', '1166', '1182', '1200', '1212', '1234', '1246', '1261', '1275', '1287', '1303', '1315', '1324', '1345', '1355', '1372', '1390', '1405', '1430', '1447', '1450', '1459', '1478', '1490', '1511', '1523', '1537', '1546', '1574', '1581', '1596', '1614', '1633', '1645', '1668', '1676', '1708', '1729', '1741', '1753', '1764', '1777', '1786', '1793', '1806', '1831', '1840', '1865', '1888', '1893', '1907', '1923', '1938', '1960', '1969', '1976', '1986', '1995', '2011', '2025', '2037', '2051', '2064', '2074', '2090', '2098', '2116', '2133', '2145', '2158', '2173', '2191', '2223', '2230', '2249', '2264', '2278', '2288', '2297', '2317', '2330', '2348', '2363', '2377', '2396', '2419', '2432', '2448', '2475', '2488', '2505', '2516', '2528', '2545', '2558', '2574', '2589', '2603', '2611', '2629', '2643', '2659', '2671', '2691', '2711', '2716', '2728', '2740', '2755', '2777', '2791', '2802', '2814', '2834', '2850', '2872', '2880', '2902', '2917', '2927', '2946', '2959', '2968', '2984', '2992', '3013', '3021', '3045', '3057', '3088', '3101', '3111', '3129', '3140', '3151', '3163', '3182', '3194', '3206', '3223', '3239', '3253', '3260', '3276', '3284', '3292', '3307', '3321', '3331', '3341', '3350', '3368', '3375', '3395', '3413', '3419', '3437', '3444', '3457', '3469', '3487', '3499', '3512', '3520', '3534', '3547', '3561', '3580', '3587', '3600', '3627', '3636', '3648', '3658', '3679', '3693', '3706', '3720', '3737', '3746', '3758', '3774', '3790', '3806', '3815', '3831', '3841', '3861', '3871', '3894', '3907', '3919', '3931', '3937', '3952', '3968', '3977', '3985', '3997', '4012', '4024', '4039', '4049', '4056', '4071', '4083', '4113', '4135', '4147', '4183', '4215', '4239', '4259', '4277', '4297', '4323', '4339', '4395', '4422', '4454', '4470', '4498', '4512', '4527', '4550', '4566', '4584', '4601', '4634', '4647', '4669', '4693', '4713', '4750', '4767', '4782', '4825', '4839', '4858', '4883', '4911', '4931', '4947', '4969', '4991', '5009', '5038', '5059', '5080', '5105', '5115', '5147', '5179', '5184', '5201', '5223', '5240', '5283', '5315', '5339', '5360', '5404', '5411', '5417', '5445', '5457', '5466', '5472', '5493', '5498', '5516', '5554', '5571', '5630', '5635', '5647', '5663', '5702', '5713', '5750', '5773', '5791', '5805', '5817', '5834', '5868', '5882', '5894', '5916', '5932', '5950', '5981', '5996', '6030', '6049', '6066', '6099', '6116', '6140', '6161', '6176', '6200', '6224', '6241', '6269', '6304', '6350', '6367', '6387', '6399', '6415', '6439', '6463', '6473', '6509', '6528', '6536', '6558', '6577', '6591', '6625', '6632', '6646', '6715', '6747', '6760', '6779', '6795', '6815', '6825', '6852', '6868', '6913', '6933', '6947', '6968', '6976', '6996', '7021', '7057', '7080', '7108', '7140', '7143', '7157', '7179', '7193', '7211', '7236', '7260', '7282', '7302', '7323', '7343', '7354', '7382', '7397', '7422', '7459', '7482', '7495', '7519', '7538', '7554', '7567', '7609', '7625', '7647', '7669', '7679', '7701', '7723', '7734', '7741', '7754', '7773', '7818', '7857', '7882', '7900', '7931', '7964', '7998', '8014', '8047', '8070', '8085', '8113', '8136', '8157', '8171', '8201', '8212', '8244', '8262', '8291', '8305', '8346', '8359', '8382', '8383', '8405', '8418', '8448', '8459', '8481', '8496', '8528', '8560', '8571', '8587', '8620', '8660', '8683', '8705', '8718', '8731', '8751', '8796', '8801', '8833', '8842', '8869', '8902', '8926', '8928', '9037', '11294', '11774', '11779', '11796', '11816', '11831', '11845', '11856', '11870', '11880', '11892', '11911', '11921', '11943', '11961', '11985', '12000', '12013', '12022', '12031', '12045', '12063', '12073', '12095', '12121', '12132', '12154', '12164', '12188', '12204', '12216', '12231', '12243', '12258', '12269', '12291', '12296', '12314', '12327', '12342', '12354', '12373', '12386', '12415', '12425', '12439', '12449', '12460', '12474', '12526', '12538', '12550', '12568', '12590', '12607', '12620', '12637', '12643', '12664', '12696', '12721', '12736', '12748', '12798', '12816', '12839', '12849', '12860', '12874', '12887', '12910', '12933', '12952', '12971', '12990', '13007', '13024', '13071', '13080', '13100', '13270', '13290', '13303', '13318', '13327', '13346', '13369', '13390', '13406', '13414', '13434', '13470', '13483', '13517', '13539', '13561', '13574', '13598', '13613', '13620', '13639', '13644', '13663', '13708', '13732', '13770', '13784', '13827', '13849', '13855', '13900', '13932', '13947', '13978', '13989', '14004', '14022', '14052', '14078', '14097', '14132', '14143', '14238', '14253', '14289', '14324', '14335', '14344', '14372', '14396', '14426', '14451', '14504', '14553', '14592', '14615', '14669', '14682', '14770', '14793', '14808', '14823', '14842', '14876', '14886', '14950', '14967', '14980', '14994', '15009', '15019', '15030', '15041', '15057', '15073', '15082', '15148', '15165', '15197', '15211', '15244', '15258', '15304', '15400', '15431', '15491', '15736', '15745', '15765', '15787', '15813', '15889', '15917', '16014', '16135', '16743', '16756', '16780', '16906', '16929', '16937', '17017', '17039', '17224', '17311', '17326', '17410', '17436', '17516', '17561', '17583', '17586', '17617', '17755', '18003', '18027']}
# blog_list = {'志田 愛佳':['57', '96', '104', '120', '127', '131', '151', '162', '180', '185', '198', '220', '229', '243', '270', '287', '289', '305', '327', '340', '357', '390', '411', '419', '425', '438', '443', '458', '476', '482', '494', '520', '536', '582', '628', '688', '708', '721', '726', '743', '775', '792', '820', '836', '877', '896', '908', '931', '951', '1020', '1041', '1071', '1092', '1131', '1149', '1163', '1177', '1207', '1224', '1269', '1297', '1323', '1334', '1401', '1453', '1499', '1519', '1578', '1624', '1637', '1654', '1706', '1723', '1733', '1770', '1790', '1805', '1858', '1875', '1912', '1929', '1952', '2018', '2050', '2085', '2120', '2159', '2170', '2183', '2215', '2241', '2256', '2271', '2307', '2337', '2358', '2373', '2408', '2424', '2443', '2455', '2457', '2477', '2489', '2537', '2586', '2620', '2654', '2667', '2700', '2751', '2774', '2809', '2847', '2864', '2923', '3017', '3040', '3105', '3120', '3136', '3174', '3203', '3266', '3294', '3342', '3384', '3402', '3417', '3466', '3493', '3519', '3543', '3558', '3597', '3643', '3687', '3689', '3729', '3752', '3803', '3812', '3826', '3859', '3872', '3920', '3922', '3938', '4021', '4034', '4078', '4136', '4192', '4199', '4218', '4225', '4264', '4283', '4384', '4409', '4467', '4565', '4586', '4622', '4676', '4734', '4843', '4871', '4894', '4915', '4965', '5024', '5070', '5137', '5207', '5229', '5250', '5297', '5302', '5372', '5439', '5558', '5596', '5745', '5812', '5874', '5948', '5955', '5972', '6016', '6036', '6071', '6113', '6137', '6191', '6217', '6235', '6262', '6282', '6293', '6318', '6380', '6452', '6487', '6548', '6560', '6567', '6655', '6668', '6696', '6716', '6741', '6764', '6788', '6839', '6880', '6922', '6959', '6991', '7084', '7124', '7127', '7148', '7169', '7248', '7264', '7286', '7292', '7329', '7372', '7389', '7405', '7444', '7451', '7472', '7509', '7527', '7593', '7638', '7692', '7832', '7860', '7975', '8018', '8094', '8123', '8146', '8148', '8231', '8292', '8370', '8543', '8708', '8735', '8781', '8957', '8981', '8986', '9045', '9069', '9134', '9209', '9515', '9519', '9593', '9691', '9822', '9862', '10026', '10260', '10412', '10535', '10681', '10792', '10795', '10937', '10985', '11616', '11650', '11658', '11847', '11877', '12092', '12128', '12140', '12338', '12611', '13072', '13337', '13423', '13529', '13614', '13733', '13872', '14162', '14268', '15048', '18188']}
# blog_list = {'米谷 奈々未':['72', '80', '105', '164', '205', '258', '293', '322', '366', '403', '431', '473', '513', '545', '568', '604', '632', '663', '693', '724', '767', '789', '807', '830', '846', '860', '875', '904', '919', '944', '962', '982', '994', '1016', '1029', '1052', '1066', '1074', '1108', '1144', '1164', '1194', '1235', '1260', '1296', '1311', '1333', '1375', '1398', '1437', '1471', '1502', '1533', '1564', '1597', '1628', '1666', '1711', '1734', '1761', '1784', '1803', '1848', '1876', '1896', '1937', '1966', '1979', '2002', '2033', '2061', '2076', '2114', '2144', '2149', '2163', '2201', '2238', '2276', '2315', '2347', '2372', '2407', '2450', '2478', '2514', '2540', '2576', '2607', '2648', '2673', '2693', '2717', '2745', '2781', '2817', '2854', '2878', '2913', '2944', '2969', '2988', '3008', '3022', '3058', '3084', '3106', '3144', '3162', '3198', '3228', '3250', '3269', '3295', '3329', '3360', '3389', '3415', '3448', '3470', '3491', '3524', '3540', '3577', '3604', '3630', '3661', '3695', '3724', '3743', '3771', '3794', '3828', '3860', '3899', '3929', '3962', '3993', '4023', '4046', '4066', '4131', '4144', '4214', '4257', '4284', '4393', '4446', '4522', '4541', '4587', '4625', '4670', '4723', '4765', '4800', '4833', '4893', '4924', '4982', '5032', '5090', '5130', '5188', '5215', '5256', '5319', '5367', '5403', '5436', '5487', '5530', '5569', '5652', '5723', '5746', '5897', '5974', '6018', '6087', '6132', '6264', '6362', '6422', '6524', '6584', '6746', '6833', '7009', '7065', '7207', '7338', '7434', '7487', '7673', '7713', '7746', '7800', '7908', '7944', '8010', '8064', '8104', '8144', '8167', '8213', '8254', '8284', '8354', '8415', '8462', '8497', '8586', '8652', '8676', '8747', '8782', '8848', '8884', '8918', '8962', '9001', '9143', '9173', '9218', '9357', '9453', '9481', '9576', '9646', '9694', '9765', '9775', '9874', '10073', '10172', '10190', '10252', '10281', '10329', '10377', '10452', '10589', '10616', '10835', '10913', '10930', '10955', '11058', '11090', '11150', '11258', '11350', '11414', '12114', '12126', '12242', '12295', '12385', '12433', '12630', '12713', '12906', '13440', '13608', '13871', '13939', '14321', '14434', '14512', '14812', '14854', '15069', '15545', '15744', '16741', '17450', '17532', '17603', '17975', '18121', '18208', '18295', '18633']}
blog_list = {'小林 由依':['69', '91', '116', '137', '176', '186', '210', '234', '254', '277', '292', '309', '323', '342', '363', '378', '395', '414', '428', '444', '461', '492', '502', '519', '539', '557', '567', '584', '601', '617', '634', '652', '671', '686', '697', '710', '725', '746', '766', '776', '794', '808', '832', '848', '862', '879', '895', '909', '936', '957', '977', '995', '1014', '1031', '1046', '1061', '1072', '1095', '1105', '1120', '1142', '1150', '1162', '1192', '1199', '1210', '1227', '1244', '1253', '1281', '1286', '1317', '1326', '1342', '1354', '1370', '1387', '1411', '1426', '1444', '1461', '1477', '1489', '1504', '1529', '1538', '1545', '1562', '1585', '1591', '1606', '1625', '1642', '1657', '1670', '1703', '1722', '1735', '1747', '1773', '1794', '1812', '1832', '1842', '1862', '1877', '1903', '1913', '1936', '1950', '1980', '1993', '2017', '2027', '2039', '2058', '2084', '2094', '2099', '2117', '2128', '2153', '2171', '2190', '2224', '2239', '2260', '2274', '2298', '2314', '2342', '2362', '2375', '2393', '2413', '2426', '2466', '2476', '2491', '2511', '2525', '2557', '2580', '2587', '2616', '2624', '2656', '2672', '2697', '2707', '2725', '2760', '2780', '2797', '2829', '2849', '2866', '2877', '2893', '2910', '2939', '2955', '2974', '2995', '3012', '3030', '3062', '3075', '3097', '3112', '3134', '3152', '3180', '3202', '3222', '3247', '3268', '3283', '3308', '3326', '3347', '3374', '3391', '3411', '3438', '3460', '3480', '3494', '3525', '3556', '3591', '3606', '3621', '3656', '3676', '3718', '3730', '3757', '3780', '3792', '3820', '3842', '3873', '3887', '3927', '3944', '3966', '3989', '4017', '4025', '4050', '4054', '4097', '4145', '4184', '4247', '4286', '4354', '4397', '4435', '4490', '4517', '4553', '4616', '4640', '4685', '4729', '4746', '4771', '4813', '4845', '4891', '4918', '4940', '4977', '5026', '5092', '5121', '5146', '5212', '5236', '5261', '5284', '5333', '5376', '5410', '5428', '5483', '5523', '5562', '5613', '5645', '5685', '5747', '5787', '5818', '5856', '5877', '5914', '5954', '5984', '6004', '6045', '6103', '6141', '6196', '6252', '6307', '6358', '6395', '6456', '6492', '6562', '6616', '6627', '6682', '6759', '6819', '6917', '6971', '7022', '7088', '7154', '7186', '7283', '7349', '7425', '7505', '7577', '7636', '7707', '7744', '7792', '7805', '7844', '7886', '8028', '8139', '8194', '8276', '8357', '8452', '8563', '8623', '8711', '8797', '8887', '8915', '8939', '8988', '9051', '9129', '9161', '9244', '9326', '9372', '9447', '9532', '9629', '9671', '9762', '9821', '9879', '10000', '10072', '10158', '10231', '10282', '10313', '10472', '10476', '10582', '10659', '10702', '10773', '10873', '11005', '11049', '11097', '11140', '11197', '11209', '11308', '11376', '11604', '11614', '11631', '11665', '11720', '11836', '11851', '11935', '12030', '12139', '12161', '12227', '12315', '12443', '12496', '12562', '12656', '12732', '12844', '12871', '12921', '13000', '13070', '13185', '13225', '13320', '13387', '13446', '13480', '13586', '13685', '13753', '13901', '14087', '14190', '14282', '14402', '14420', '14459', '14533', '14610', '14709', '14840', '14880', '14953', '15098', '15218', '15279', '15474', '15605', '15762', '15951', '16168', '16374', '16444', '16691', '16881', '17074', '17289', '17432', '17782', '17849', '17897', '17986', '18183', '18351', '18423', '18565']}




In [447]:
# Main

start = time.time()
Menber_kanji,Menber_hiragana,Menber_name = set_Name_of_the_Menber()

# blog_list,latest_blog_num = get_Number_of_Blog(Menber_kanji,Menber_hiragana)
# make_txt_file(blog_list,latest_blog_num)


# latest_blog_num = get_Number_of_LatestBlog()
# blog_list = get_List_from_Textfile()

GetPicture(blog_list[Menber_name],Menber_name)
# get_Picture_from_Blog(blog_num)
# folder_directory=make_Folder_for_save_Picture(name)

elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")



0: 石森 虹花
1: 今泉 佑唯
2: 上村 莉菜
3: 尾関 梨香
4: 織田 奈那
5: 小池 美波
6: 小林 由依
7: 齋藤 冬優花
8: 佐藤 詩織
9: 志田 愛佳
10: 菅井 友香
11: 鈴本 美愉
12: 長沢 菜々香
13: 長濱 ねる
14: 土生 瑞穂
15: 原田 葵
16: 平手 友梨奈
17: 守屋 茜
18: 米谷 奈々未
19: 渡辺 梨加
20: 渡邉 理佐
21: 井口 眞緒
22: 潮 紗理菜
23: 柿崎 芽実
24: 影山 優佳
25: 加藤 史帆
26: 齊藤 京子
27: 佐々木 久美
28: 佐々木 美玲
29: 高瀬 愛奈
30: 高本 彩花
31: 東村 芽依
32: 金村 美玖
33: 河田 陽菜
34: 小坂 菜緒
35: 富田 鈴花
36: 丹生 明里
37: 濱岸 ひより
38: 松田 好花
39: 宮田 愛萌
40: 渡邉 美穂


KeyboardInterrupt: 

In [ ]:
f = open('list100.txt')
data1 = f.read()  # ファイル終端まで全て読んだデータを返す
f.close()

lines1 = data1.split('\n') # 改行で区切る(改行文字そのものは戻り値のデータには含まれない)
for line in lines1:
    print(line)

print(len(line))